In [84]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from utils.gtsrb_dataset import GTSRBDataset
from models.tsr_cnn import TSRNet
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm 

In [97]:
from utils.train_tsr import train_tsr
from models.tsr_cnn import TSRNet
from torch.utils.data import DataLoader
from torchvision import transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets & loaders
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = GTSRBDataset(csv_file="./data/Train.csv", root_dir="./data/", transform=transform)
test_dataset  = GTSRBDataset(csv_file="./data/Test.csv",  root_dir="./data/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Train samples: {len(train_dataset)} | Test samples: {len(test_dataset)}")
# Model
model = TSRNet(num_classes=43).to(device)

# Train
model, best_acc = train_tsr(model, train_loader, test_loader, device, num_epochs=5)


Train samples: 39209 | Test samples: 12630


Epoch 1/5: 100%|██████████| 613/613 [00:13<00:00, 46.34it/s]


Epoch [1/5] Loss: 0.5424 | Train Acc: 85.05%
 Test Accuracy: 90.36%
✅ Saved checkpoint: ./results/checkpoints/tsr_epoch1_acc90.pth


Epoch 2/5: 100%|██████████| 613/613 [00:14<00:00, 42.47it/s]


Epoch [2/5] Loss: 0.0631 | Train Acc: 98.34%
 Test Accuracy: 92.87%
✅ Saved checkpoint: ./results/checkpoints/tsr_epoch2_acc92.pth


...

Epoch [3/5] Loss: 0.0301 | Train Acc: 99.16%
 Test Accuracy: 92.88%
✅ Saved checkpoint: ./results/checkpoints/tsr_epoch3_acc92.pth


Epoch 4/5: 100%|██████████| 613/613 [00:12<00:00, 47.42it/s]


Epoch [4/5] Loss: 0.0247 | Train Acc: 99.25%
 Test Accuracy: 93.85%
✅ Saved checkpoint: ./results/checkpoints/tsr_epoch4_acc93.pth


Epoch 5/5: 100%|██████████| 613/613 [00:11<00:00, 54.87it/s]


Epoch [5/5] Loss: 0.0139 | Train Acc: 99.60%
 Test Accuracy: 90.91%
Training complete. Best accuracy: 93.85%


In [98]:
from models.tsr_cnn import TSRNet
from utils.attack_eval import evaluate_clean, evaluate_fgsm, evaluate_pgd, evaluate_patch
from utils.attacks import train_adversarial_patch
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load TSR model
model = TSRNet(num_classes=43).to(device)
ckpt = torch.load("results/checkpoints/tsr_best_model.pth", map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

# Evaluate clean accuracy
clean_acc = evaluate_clean(model, test_loader, device)
print("Clean Accuracy:", clean_acc)

# FGSM
eps = 0.03
adv_acc, asr = evaluate_fgsm(model, test_loader, device, eps)
print(f"FGSM Adv Acc: {adv_acc:.2f}%, ASR: {asr:.2f}%")

# PGD
adv_acc, asr = evaluate_pgd(model, test_loader, device, eps, alpha=0.007, iters=10)
print(f"PGD Adv Acc: {adv_acc:.2f}%, ASR: {asr:.2f}%")

# Train and evaluate patch attack
patch = train_adversarial_patch(model, train_loader, device, num_epochs=1, patch_size=0.18, lr=0.05)
res = evaluate_patch(model, test_loader, patch, device, patch_size=0.18)
print("Patch attack eval:", res)


Clean Accuracy: 92.62866191607284
FGSM Adv Acc: 42.60%, ASR: 54.00%
PGD Adv Acc: 37.64%, ASR: 59.36%
[Patch train] Epoch 1/1, loss: -2.3790
Patch attack eval: {'clean_acc': 92.62866191607284, 'adv_acc': 71.84481393507522, 'asr': 23.52337806650141}


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from models.autoencoder import Autoencoder, save_checkpoint  # your file

# ----------------------------
# 1. Configuration
# ----------------------------
batch_size = 64
num_epochs = 5
latent_dim = 128
lr = 1e-3

# ----------------------------
# 2. Dataset & Dataloader
# ----------------------------


# ----------------------------
# 3. Model, Loss, Optimizer
# ----------------------------
model = Autoencoder(latent_dim=latent_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

ckpt_dir = "results/checkpoints/autoencoder"
os.makedirs(ckpt_dir, exist_ok=True)
best_test_loss = float("inf")

# -----------------------------
# Training loop
# -----------------------------


for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for imgs, _ in train_loader:
        imgs = imgs.to(device)
        optimizer.zero_grad()
        z, x_rec = model(imgs)
        loss = criterion(x_rec, imgs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    
    train_loss = running_loss / len(train_loader.dataset)

    # -----------------------------
    # Evaluation on test set
    # -----------------------------
    model.eval()
    test_loss_total = 0
    with torch.no_grad():
        for imgs, _ in test_loader:
            imgs = imgs.to(device)
            z, x_rec = model(imgs)
            loss = criterion(x_rec, imgs)
            test_loss_total += loss.item() * imgs.size(0)
    test_loss = test_loss_total / len(test_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # -----------------------------
    # Save checkpoint if best
    # -----------------------------
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        ckpt_name = os.path.join(ckpt_dir, f"autoencoder_best.pth")
        save_checkpoint(model, optimizer, epoch+1, ckpt_name)
        print(f"✅ Saved best checkpoint at epoch {epoch+1}")

print("🎯 Training completed!")

Epoch [1/5] | Train Loss: 0.0587 | Test Loss: 0.0282
✅ Saved best checkpoint at epoch 1
Epoch [2/5] | Train Loss: 0.0241 | Test Loss: 0.0200
✅ Saved best checkpoint at epoch 2
Epoch [3/5] | Train Loss: 0.0177 | Test Loss: 0.0162
✅ Saved best checkpoint at epoch 3
Epoch [4/5] | Train Loss: 0.0146 | Test Loss: 0.0154
✅ Saved best checkpoint at epoch 4
Epoch [5/5] | Train Loss: 0.0131 | Test Loss: 0.0124
✅ Saved best checkpoint at epoch 5
🎯 Training completed!


In [63]:
# train_unet_watermark.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
import os
from models.autoencoder import Autoencoder
from models.unet_encoder import UNetWatermark


# -----------------------------
# Load pretrained Autoencoder
# -----------------------------
latent_dim = 128
autoencoder = Autoencoder(latent_dim=latent_dim).to(device)
ae_ckpt = torch.load("results/checkpoints/autoencoder/autoencoder_best.pth", map_location=device)
autoencoder.load_state_dict(ae_ckpt["model_state"])
autoencoder.eval()
for param in autoencoder.parameters():
    param.requires_grad = False

# -----------------------------
# Initialize UNet
# -----------------------------
unet = UNetWatermark(latent_dim=latent_dim).to(device)
optimizer = optim.Adam(unet.parameters(), lr=1e-4)

# -----------------------------
# VGG perceptual loss
# -----------------------------
vgg = models.vgg19(pretrained=True).features[:16].to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

def perceptual_loss(I, Iw):
    f1 = vgg(I)
    f2 = vgg(Iw)
    return torch.mean((f1 - f2)**2)

# -----------------------------
# Checkpoint folder
# -----------------------------
os.makedirs("checkpoints", exist_ok=True)

# -----------------------------
# Training loop
# -----------------------------
num_epochs = 10

for epoch in range(num_epochs):
    unet.train()
    running_loss = 0
    for imgs, _ in train_loader:
        imgs = imgs.to(device)

        # Get latent vector from pretrained autoencoder
        with torch.no_grad():
            z, _ = autoencoder(imgs)
        print("Latent z shape:", z.shape)
        # Generate watermarked image
        Iw = unet(imgs, z)

        # Compute perceptual loss
        loss = perceptual_loss(imgs, Iw)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] | Perceptual Loss: {avg_loss:.6f}")

    # Save checkpoint
    torch.save(unet.state_dict(), f"checkpoints/unet_epoch_{epoch+1}.pth")
    print(f"✅ Saved checkpoint for epoch {epoch+1}")

print("🎯 UNet watermark training complete!")


/home/deepesh/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/deepesh/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x8192 and 2048x128)